In [ ]:
import zmq
import numpy as np
import skimage as ski
import matplotlib.pyplot as plt
import scipy as sp

In [ ]:
PORT = "7878"
DOMAIN = "*"
SOCKET_ADDR = f"tcp://{DOMAIN}:{PORT}"

In [ ]:
HELLO = "Hello"
ACK = "Acknowledge"
DENIED = "Denied"

In [ ]:
context = zmq.Context()
socket = context.socket(zmq.PAIR)
socket.copy_threshold = 0
b = socket.bind(SOCKET_ADDR)
b

In [ ]:
# handshake
socket.send_string(HELLO)
print("Sent hello, waiting for acknowledgement...")
ack = socket.recv_string()
if ack == ACK:
    print('Received connection ack:', ack)
else:
    print("Received unkown message", ack)

In [ ]:
header = socket.recv_json()
header

In [ ]:
# acknowledge receipt of header, ask for image data
socket.send_string(ACK)

In [ ]:
im_bytes = socket.recv(copy=False)
im_bytes

In [ ]:
buf = memoryview(im_bytes)
im = np.frombuffer(buf, dtype=header['descr'])
im = (im * 255).astype(np.uint8)
im.shape = header['shape']
im

In [ ]:
# acknowledge receipt of image data
socket.send_string(ACK)

In [ ]:
plt.imshow(im, cmap='gray')

In [ ]:
import napari

In [ ]:
viewer = napari.view_image(im)

In [ ]:
# Create the histogram
plt.figure(figsize=(10, 6))
plt.hist(im.flatten(), bins=256, range=(0, 256), edgecolor='black')

# Labeling the axes and adding a title
plt.xlabel('Pixel Value')
plt.ylabel('Frequency')
plt.title('Pixel Value Frequency Distribution')

# Display the plot
plt.show()

In [ ]:
print(plt.colormaps())

In [ ]:
cmap_name = 'Paired'
# Reshape to a 2D array (gradient image) with shape (1, 256)
gradient = np.arange(256)[np.newaxis, :]

# Display the gradient image using the specified colormap
plt.figure(figsize=(8, 2))
plt.imshow(gradient, aspect='auto', cmap=cmap_name)
plt.colorbar(label='Pixel Value')
plt.title(f'Colormap: {cmap_name}')
plt.xlabel('Value')
plt.ylabel('Intensity')
plt.yticks([])
plt.show()

In [ ]:
ets = lambda v: int(v/65535*255)
ste = lambda v: int(v/255*65535)

In [ ]:
markers = np.zeros_like(im, dtype=np.uint8)
IDK = 0
BG = 1
FG = 2
# markers[im < ste(30)] = BG # blue
# markers[im > ste(50)] = FG # brown
markers[im < 30] = BG # blue
markers[im > 50] = FG # brown
# IDK = RED

# plt.imshow(np.array(markers / markers.max() * (2**8-1), dtype=np.uint8), cmap='gray')
plt.imshow(np.array(markers / markers.max() * (2**8-1), dtype=np.uint8), cmap=cmap_name)

In [ ]:
elevation_map = ski.filters.sobel(im)

plt.imshow(elevation_map)

In [ ]:
segmentation = ski.segmentation.watershed(elevation_map, markers)
segmentation = sp.ndimage.binary_fill_holes(segmentation - 1)

plt.imshow(segmentation)

In [ ]:
# Create a figure with two subplots side by side
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

# Display the first image in the first subplot
axes[0].imshow(im, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('Orig')
axes[0].axis('off')  # Hide the axes

# Display the second image in the second subplot
axes[1].imshow(segmentation, cmap=cmap_name, vmin=0, vmax=segmentation.max())
axes[1].set_title('Seg')
axes[1].axis('off')  # Hide the axes

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

In [ ]:
labels, _ = sp.ndimage.label(segmentation)

# remove small objects
sizes = np.bincount(labels.ravel())
mask_sizes = sizes > 20
mask_sizes[0] = 0
segmentation = mask_sizes[labels]

labels, _ = sp.ndimage.label(segmentation)

In [ ]:
plt.imshow(labels)

In [ ]:
return_header = np.lib.format.header_data_from_array_1_0(labels)
return_header

In [ ]:
socket.send_json(return_header)

In [ ]:
ack = socket.recv_string()
if ack == ACK:
    print('Received return header ack:', ack)
else:
    print("Received unkown message", ack)

In [ ]:
socket.send(labels, copy=False)

In [ ]:
# socket.send_string("Cancel")